In [1]:
import QuantLib as ql
import openpyxl as xl

In [2]:
def DatetimeToDate(ADatetime):
    ADate = ql.Date(ADatetime.day, ADatetime.month, ADatetime.year)
    return ADate

In [3]:
class DCrossCurrencySwap:    
    def __init__(self, evDate, effective, maturity, position, CCYComm, CommDC, 
                 CommIdx, CommFreq, CommSpread, InitCommNPA, FinalCommNPA, 
                 CCYTerm, TermDC, TermFreq, TermRate, InitTermNPA, FinalTermNPA, 
                 FXSpot, h_discTSComm, h_discTSTerm):
        self._ValueDate = evDate
        self._EffectiveDate = effective
        self._Maturity = maturity        
        if position == 'Long':          
            self._Position = 1.0
        else:
            self._Position = -1.0

        self._CCYComm = CCYComm
        self._CommDC = CommDC
        self._CommIdx = CommIdx
        self._InitCommNPA = InitCommNPA
        self._FinalCommNPA = FinalCommNPA
        self._CommSpread = CommSpread
        self._CommFreq = CommFreq

        self._CCYTerm = CCYTerm
        self._TermDC = TermDC
        self._InitTermNPA = InitTermNPA
        self._FinalTermNPA = FinalTermNPA
        self._TermRate = TermRate
        self._TermFreq = TermFreq

        self._FXSpot = FXSpot
        self._H_discTSComm = h_discTSComm
        self._H_discTSTerm = h_discTSTerm

    def NPV(self):
        _DGR = ql.DateGeneration.Backward
        _BDC = ql.Unadjusted
        _Cal = ql.NullCalendar()
        schTerm = ql.Schedule(self._EffectiveDate, self._Maturity, 
                              ql.Period(3, ql.Months), _Cal, _BDC, _BDC, _DGR, False)
        tenor = int((self._Maturity.year() - self._EffectiveDate.year()) 
                    * 12 / int(self._TermFreq))
        ListCoupon = list()
        for j in range(tenor):
            ListCoupon.append(self._TermRate)
            
        VecCoupon = ql.DoubleVector(ListCoupon)
        TermFaceAmt = self._FinalTermNPA
        TermBond = ql.FixedRateBond(0, TermFaceAmt, schTerm, VecCoupon, self._TermDC, 
                                    BDC, TermFaceAmt)
        TermBondEngine = ql.DiscountingBondEngine(self._H_discTSTerm)
        TermBond.setPricingEngine(TermBondEngine)
        TermValue = TermBond.dirtyPrice()
        
        schComm = ql.Schedule(self._EffectiveDate, self._Maturity, 
                              ql.Period(3, ql.Months), _Cal, _BDC, _BDC, _DGR, False)
        tenor = int((self._Maturity.year() - self._EffectiveDate.year()) 
                    * 12 / int(self._CommFreq))
        ListSpread = list()
        ListGearing = list()
        for j in range(tenor):
            ListSpread.append(self._CommSpread)
            ListGearing.append(1.0)
        VecSpread = ql.DoubleVector(ListSpread)
        VecGearing = ql.DoubleVector(ListGearing)
        CommFaceAmt = self._FinalCommNPA
        CommBond = ql.FloatingRateBond(0, CommFaceAmt, schComm, self._CommIdx, 
                                       self._CommDC, _BDC, 2, VecGearing, VecSpread)
        CommBondEngine = ql.DiscountingBondEngine(self._H_discTSComm)
        CommBond.setPricingEngine(CommBondEngine)
        CommValue = CommBond.dirtyPrice()
        
        if(evDate < self._EffectiveDate):
            dfComm = self._H_discTSComm.discount(self._EffectiveDate)
            dfTerm = self._H_discTSTerm.discount(self._EffectiveDate)
            initValue = (self._InitTermNPA * dfTerm 
                         - self._FXSpot * self._InitCommNPA * dfComm)
            self._Value = self._Position * (CommValue * self._FXSpot - TermValue 
                                            + initValue)
        else:
            self._Value = self._Position * (CommValue * self._FXSpot - TermValue)
        return self._Value

In [4]:
PstBook = xl.load_workbook('Pst_CCS.xlsx', data_only=True)
MktSheet = PstBook.worksheets[0]
CommCCY = 'USD'
TermCCY = 'TWD'
Type = 'CCS'
MTMDate = MktSheet.cell(1, 3).value
FXSpot = MktSheet.cell(3, 6).value

In [5]:
settings = ql.Settings.instance()
evDate = DatetimeToDate(MTMDate)
settings.setEvaluationDate(evDate)
Cal = ql.NullCalendar()
CommDC = ql.Actual360()
TermDC = ql.Actual365Fixed()
BDC = ql.Unadjusted
settlementDays = 0

In [6]:
TermZeroSP = MktSheet.cell(43, 5).value
TermZero3M = MktSheet.cell(44, 5).value
TermZero6M = MktSheet.cell(45, 5).value
TermZero1Y = MktSheet.cell(46, 5).value
TermZero2Y = MktSheet.cell(47, 5).value
TermZero3Y = MktSheet.cell(48, 5).value
TermZero5Y = MktSheet.cell(49, 5).value
TermZero10Y = MktSheet.cell(50, 5).value
TermZero15Y = MktSheet.cell(51, 5).value
TermZero20Y = MktSheet.cell(52, 5).value
TermZero30Y = MktSheet.cell(53, 5).value
print(TermZeroSP)

0.004795999997252601


In [7]:
DateSP = evDate
Date3M = Cal.advance(evDate, 3, ql.Months, BDC, False)
Date6M = Cal.advance(evDate, 6, ql.Months, BDC, False)
Date1Y = Cal.advance(evDate, 1, ql.Years, BDC, False)
Date2Y = Cal.advance(evDate, 2, ql.Years, BDC, False)
Date3Y = Cal.advance(evDate, 3, ql.Years, BDC, False)
Date5Y = Cal.advance(evDate, 5, ql.Years, BDC, False)
Date10Y = Cal.advance(evDate, 10, ql.Years, BDC, False)
Date15Y = Cal.advance(evDate, 15, ql.Years, BDC, False)
Date20Y = Cal.advance(evDate, 20, ql.Years, BDC, False)
Date30Y = Cal.advance(evDate, 30, ql.Years, BDC, False)
print(DateSP)

June 30th, 2021


In [8]:
CommZeroSP = MktSheet.cell(43, 25).value
CommZero3M = MktSheet.cell(44, 25).value
CommZero6M = MktSheet.cell(45, 25).value
CommZero1Y = MktSheet.cell(46, 25).value
CommZero2Y = MktSheet.cell(47, 25).value
CommZero3Y = MktSheet.cell(48, 25).value
CommZero5Y = MktSheet.cell(49, 25).value
CommZero10Y = MktSheet.cell(50, 25).value
CommZero15Y = MktSheet.cell(51, 25).value
CommZero20Y = MktSheet.cell(52, 25).value
CommZero30Y = MktSheet.cell(53, 25).value
print(CommZeroSP)

0.001459727696536763


In [9]:
ListDate = list([DateSP, Date3M, Date6M, Date1Y, Date2Y, Date3Y, Date5Y, Date10Y, 
                 Date15Y, Date20Y, Date30Y])
ListTermRate = list([TermZeroSP, TermZero3M, TermZero6M, TermZero1Y, TermZero2Y, 
                     TermZero3Y, TermZero5Y, TermZero10Y, TermZero15Y, TermZero20Y, 
                     TermZero30Y])
ListCommRate = list([CommZeroSP, CommZero3M, CommZero6M, CommZero1Y, CommZero2Y, 
                     CommZero3Y, CommZero5Y, CommZero10Y, CommZero15Y, CommZero20Y, 
                     CommZero30Y])
ListTermRate

[0.004795999997252601,
 0.004795999997762682,
 0.005679705459626832,
 0.005055409226605943,
 0.005392645814005983,
 0.005923963111383993,
 0.00685202799427061,
 0.00789843471265416,
 0.008384574081848417,
 0.00862747768539673,
 0.008870403457750699]

In [10]:
VecDate = ql.DateVector(ListDate)
VecTermRate = ql.DoubleVector(ListTermRate)
VecCommRate = ql.DoubleVector(ListCommRate)

In [11]:
TSTermZero = ql.ZeroCurve(VecDate, VecTermRate, TermDC, Cal, ql.Linear(), 
                          ql.Continuous, ql.Annual)
h_TSTermZero = ql.YieldTermStructureHandle(TSTermZero)
TSCommZero = ql.ZeroCurve(VecDate, VecCommRate, CommDC, Cal, ql.Linear(),
                          ql.Continuous, ql.Annual)
h_TSCommZero = ql.YieldTermStructureHandle(TSCommZero)

In [12]:
P3M = ql.Period(3, ql.Months)
P6M = ql.Period(6, ql.Months)
USD = ql.USDCurrency()
EOM = False
FixingDays = 2
USDLIBOR3M = ql.IborIndex("LIBOR3M", P3M, FixingDays, USD, Cal, BDC, EOM, CommDC, h_TSCommZero)
USDLIBOR6M = ql.IborIndex("LIBOR6M", P6M, FixingDays, USD, Cal, BDC, EOM, CommDC, h_TSCommZero)

In [13]:
L3MFixingDates = list()
L3MFixingRates = list()
for i in range(33, 131):
    FixDate = DatetimeToDate(MktSheet.cell(i, 29).value)
    FixRate = MktSheet.cell(i, 30).value / 100.0
    L3MFixingDates.append(FixDate)
    L3MFixingRates.append(FixRate)

In [14]:
L6MFixingDates = list()
L6MFixingRates = list()
for i in range(33, 221):
    FixDate = DatetimeToDate(MktSheet.cell(i, 32).value)
    FixRate = MktSheet.cell(i, 33).value / 100.0
    L6MFixingDates.append(FixDate)
    L6MFixingRates.append(FixRate)

In [15]:
L3MVecFixingDates = ql.DateVector(L3MFixingDates)
L3MVecFixingRates = ql.DoubleVector(L3MFixingRates)
USDLIBOR3M.addFixings(L3MVecFixingDates, L3MVecFixingRates, True)

In [16]:
L6MVecFixingDates = ql.DateVector(L6MFixingDates)
L6MVecFixingRates = ql.DoubleVector(L6MFixingRates)
USDLIBOR6M.addFixings(L6MVecFixingDates, L6MVecFixingRates, True)

In [17]:
CCSSheet = PstBook['CCS']
start_row = 2
end_row = 4
start_col = 0
end_col = 14
recordcount = end_col - start_col + 1
ListCCS = list()
for i in range(start_row, end_row + 1):
    row = [cell.value for cell in CCSSheet[i][start_col:end_col+1]]
    ListCCS.append(row)
    print(row)

['CCS001', datetime.datetime(2021, 6, 30, 0, 0), datetime.datetime(2021, 7, 2, 0, 0), datetime.datetime(2023, 7, 2, 0, 0), 'Long', 'TWD', 27891000, 28000000, 3, 0.0055, 'USD', 1000000, 1000000, 3, 0]
['CCS002', datetime.datetime(2021, 6, 30, 0, 0), datetime.datetime(2021, 7, 2, 0, 0), datetime.datetime(2025, 7, 2, 0, 0), 'Short', 'TWD', 27891000, 28500000, 3, 0.006, 'USD', 2000000, 2000000, 3, 0]
['CCS003', datetime.datetime(2021, 6, 30, 0, 0), datetime.datetime(2021, 7, 2, 0, 0), datetime.datetime(2025, 7, 2, 0, 0), 'Long', 'TWD', 27891000, 28750000, 3, 0.0065, 'USD', 1500000, 1500000, 3, 0]


In [18]:
CCSBook = list()
for i in range(len(ListCCS)):
    row = ListCCS[i]
    record = list()
    record.append(row[0])
    record.append(DatetimeToDate(row[1]))
    record.append(DatetimeToDate(row[2]))
    record.append(DatetimeToDate(row[3]))
    record.append(row[4])
    record.append(row[5])
    record.append(row[6])
    record.append(row[7])
    record.append(row[8])
    record.append(row[9])
    record.append(row[10])
    record.append(row[11])
    record.append(row[12])
    record.append(row[13])
    record.append(row[14])
    CCSBook.append(record)

In [19]:
CCSBook

[['CCS001',
  Date(30,6,2021),
  Date(2,7,2021),
  Date(2,7,2023),
  'Long',
  'TWD',
  27891000,
  28000000,
  3,
  0.0055,
  'USD',
  1000000,
  1000000,
  3,
  0],
 ['CCS002',
  Date(30,6,2021),
  Date(2,7,2021),
  Date(2,7,2025),
  'Short',
  'TWD',
  27891000,
  28500000,
  3,
  0.006,
  'USD',
  2000000,
  2000000,
  3,
  0],
 ['CCS003',
  Date(30,6,2021),
  Date(2,7,2021),
  Date(2,7,2025),
  'Long',
  'TWD',
  27891000,
  28750000,
  3,
  0.0065,
  'USD',
  1500000,
  1500000,
  3,
  0]]

In [20]:
SenSheet = PstBook['FXR']
for i in range(len(CCSBook)):
    record = CCSBook[i]
    effectivedate = record[2]
    maturitydate = record[3]
    position = record[4]
    commccy = record[10]
    commfreq = record[13]
    commspread = record[14]
    initcommnpa = record[11]
    finalcommnpa = record[12]
    termccy = record[5]
    termfreq = record[8]
    termrate = record[9]
    inittermnpa = record[6]
    finaltermnpa = record[7]
    CCS = DCrossCurrencySwap(evDate, effectivedate, maturitydate, position, commccy, 
                             CommDC, USDLIBOR3M, commfreq, commspread, 
                             initcommnpa, finalcommnpa, termccy, TermDC, 
                             termfreq, termrate, inittermnpa, finaltermnpa, 
                             FXSpot, h_TSCommZero, h_TSTermZero)        
    Value = CCS.NPV()
    print(Value)
    
    FXSpot_1 = FXSpot * 1.01
    CCS = DCrossCurrencySwap(evDate, effectivedate, maturitydate, position, commccy, 
                             CommDC, USDLIBOR3M, commfreq, commspread, 
                             initcommnpa, finalcommnpa, termccy, TermDC, 
                             termfreq, termrate, inittermnpa, finaltermnpa, 
                             FXSpot_1, h_TSCommZero, h_TSTermZero)    
    Value_1 = CCS.NPV()
    Delta = (Value_1 - Value) * 100.0
    print(Value_1)
    
    FXSpot_Up = FXSpot * 1.15
    CCS = DCrossCurrencySwap(evDate, effectivedate, maturitydate, position, commccy, 
                             CommDC, USDLIBOR3M, commfreq, commspread, 
                             initcommnpa, finalcommnpa, termccy, TermDC, 
                             termfreq, termrate, inittermnpa, finaltermnpa, 
                             FXSpot_Up, h_TSCommZero, h_TSTermZero)            
    Value_Up = CCS.NPV()
    print(Value_Up)
    
    FXSpot_Dn = FXSpot * 0.85
    CCS = DCrossCurrencySwap(evDate, effectivedate, maturitydate, position, commccy, 
                             CommDC, USDLIBOR3M, commfreq, commspread, 
                             initcommnpa, finalcommnpa, termccy, TermDC, 
                             termfreq, termrate, inittermnpa, finaltermnpa, 
                             FXSpot_Dn, h_TSCommZero, h_TSTermZero)    
    Value_Dn = CCS.NPV()
    print(Value_Dn)
    
    Curv_Up = (-((Value_Up - Value) - 0.15 * Delta))
    Curv_Dn = (-((Value_Dn - Value) + 0.15 * Delta))    
    
    SenSheet.cell(i+5, 1).value = record[0]
    SenSheet.cell(i+5, 2).value = Value
    SenSheet.cell(i+5, 3).value = CommCCY
    SenSheet.cell(i+5, 4).value = Delta
    SenSheet.cell(i+5, 6).value = Curv_Up
    SenSheet.cell(i+5, 8).value = Curv_Dn
    print(Delta, Curv_Up, Curv_Dn)
    
PstBook.save('Pst_CCS.xlsx')
PstBook.close()

-27696374.575095322
-27975254.43287391
-31879572.441774204
-23513176.70841644
-27887985.777858645 8.568167686462402e-08 -8.568167686462402e-08
55667746.87216
56225534.4680997
64034560.81125547
47300932.93306454
55778759.59396958 -2.9802322387695312e-08 2.2351741790771484e-08
-41966037.81051023
-42384371.53736938
-48241043.713397406
-35691031.90762305
-41833372.68591449 -0.0 -7.450580596923828e-09
